In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import weather_api_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Iqaluit,CA,2021-11-16 02:00:14,63.7506,-68.5145,0.70,64,8,15.99
1,1,Provideniya,RU,2021-11-16 02:03:01,64.3833,-173.3000,19.51,73,90,15.66
2,2,Tuatapere,NZ,2021-11-16 02:03:01,-46.1333,167.6833,51.62,72,100,11.79
3,3,Ribeira Grande,PT,2021-11-16 02:03:01,38.5167,-28.7000,68.04,83,10,14.92
4,4,Half Moon Bay,US,2021-11-16 02:01:33,37.4636,-122.4286,62.62,90,20,1.01


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=weather_api_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])                          
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
15,15,Vao,NC,2021-11-16 02:03:06,-22.6667,167.4833,76.78,71,1,11.72
27,27,Vaini,TO,2021-11-16 02:03:09,-21.2000,-175.2000,78.96,83,40,11.50
28,28,Kahului,US,2021-11-16 02:03:10,20.8947,-156.4700,82.98,81,40,12.66
29,29,Avarua,CK,2021-11-16 02:01:24,-21.2078,-159.7750,77.05,78,20,4.61
39,39,Diourbel,SN,2021-11-16 02:03:13,14.6628,-16.3925,77.81,61,90,9.42
47,47,Vaitape,PF,2021-11-16 02:04:16,-16.5167,-151.7500,80.76,74,100,10.22
50,50,Atuona,PF,2021-11-16 02:04:17,-9.8000,-139.0333,78.24,74,27,12.37
55,55,Hilo,US,2021-11-16 02:04:19,19.7297,-155.0900,83.28,86,90,8.05
64,64,Bambous Virieux,MU,2021-11-16 02:04:22,-20.3428,57.7575,76.21,70,78,10.56
67,67,Aquiraz,BR,2021-11-16 02:04:24,-3.9014,-38.3911,86.22,64,76,15.01


In [9]:
preferred_cities_df.count()

City_ID       134
City          134
Country       134
Date          134
Lat           134
Lng           134
Max Temp      134
Humidity      134
Cloudiness    134
Wind Speed    134
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
15,Vao,NC,76.78,-22.6667,167.4833,
27,Vaini,TO,78.96,-21.2000,-175.2000,
28,Kahului,US,82.98,20.8947,-156.4700,
29,Avarua,CK,77.05,-21.2078,-159.7750,
39,Diourbel,SN,77.81,14.6628,-16.3925,
47,Vaitape,PF,80.76,-16.5167,-151.7500,
50,Atuona,PF,78.24,-9.8000,-139.0333,
55,Hilo,US,83.28,19.7297,-155.0900,
64,Bambous Virieux,MU,76.21,-20.3428,57.7575,
67,Aquiraz,BR,86.22,-3.9014,-38.3911,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": weather_api_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
15,Vao,NC,76.78,-22.6667,167.4833,Hotel Kou-Bugny
27,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
28,Kahului,US,82.98,20.8947,-156.4700,Maui Seaside Hotel
29,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn
39,Diourbel,SN,77.81,14.6628,-16.3925,ISFAR maison Blanche


In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



KeyError: 'column1'

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_info' is not defined